In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 7.4MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Vader
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as sia 
df = pd.read_csv('/content/drive/MyDrive/Project/Datasets/GrammarandProductReviews_1.csv')
positivity=[]
negativity=[]
overall_sentiment=[]

sid_obj=sia()

for review in df["reviews"]:
  sentiment_dict = sid_obj.polarity_scores(review)
  positivity.append(sentiment_dict['pos'])
  negativity.append(sentiment_dict['neg'])
  overall_sentiment.append('pos' if sentiment_dict['pos']> sentiment_dict['neg'] else 'neg' if sentiment_dict['pos']<sentiment_dict['neg'] else 'pos')

df.insert(2,'Pos',positivity)
df.insert(3,'Neg',negativity)
df.insert(4,'Senti',overall_sentiment)

In [5]:
df['Review'] = df['reviews']

In [6]:
df['Review']= [x.lower() for x in df['Review']]
df['Review'] = df['Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [8]:
tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
df = df.astype({"Review":str})
tokenizer.fit_on_texts(df['Review'].values)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [10]:
X = tokenizer.texts_to_sequences(df['Review'].values)
X = pad_sequences(X)

In [11]:

embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 854, 128)          256000    
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
Y = pd.get_dummies(df['Senti']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(18396, 854) (18396, 2)
(4600, 854) (4600, 2)


In [13]:
model.fit(X_train, Y_train, batch_size = batch_size, verbose =2 )

575/575 - 1266s - loss: 0.2135 - accuracy: 0.9329


In [14]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

144/144 - 84s - loss: 0.1633 - accuracy: 0.9398
Score: 0.16
Validation Accuracy: 0.94
